In [105]:
import pandas as pd
df = pd.read_stata('nes5200_processed_voters_realideo.dta')
df.head()

,year,resid,weight1,weight2,weight3,age,gender,race,educ1,urban,...,perfin1,perfin2,perfin,presadm,age_10,age_sq_10,newfathe,newmoth,parent_party,white
0,1948.0,1001.0,1.0,1.0,1.0,NaN,1. male,1. white,1. grade school of less (0-8 grades),NaN,...,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0
1,1948.0,1002.0,1.0,1.0,1.0,NaN,2. female,1. white,"2. high school (12 grades or fewer, incl",NaN,...,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0
2,1948.0,1003.0,1.0,1.0,1.0,NaN,2. female,1. white,"2. high school (12 grades or fewer, incl",NaN,...,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0
3,1948.0,1004.0,1.0,1.0,1.0,NaN,2. female,1. white,"3. some college(13 grades or more,but no",NaN,...,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0
4,1948.0,1005.0,1.0,1.0,1.0,NaN,1. male,1. white,"3. some college(13 grades or more,but no",NaN,...,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0


In [106]:
df.columns

Index(['year', 'resid', 'weight1', 'weight2', 'weight3', 'age', 'gender',
       'race', 'educ1', 'urban', 'region', 'income', 'occup1', 'union',
       'religion', 'educ2', 'educ3', 'martial_status', 'occup2', 'icpsr_cty',
       'fips_cty', 'partyid7', 'partyid3', 'partyid3_b', 'str_partyid',
       'father_party', 'mother_party', 'dlikes', 'rlikes', 'dem_therm',
       'rep_therm', 'regis', 'vote', 'regisvote', 'presvote',
       'presvote_2party', 'presvote_intent', 'ideo_feel', 'ideo7', 'ideo',
       'cd', 'state', 'inter_pre', 'inter_post', 'black', 'female', 'age_sq',
       'rep_presvote', 'rep_pres_intent', 'south', 'real_ideo', 'presapprov',
       'perfin1', 'perfin2', 'perfin', 'presadm', 'age_10', 'age_sq_10',
       'newfathe', 'newmoth', 'parent_party', 'white'],
      dtype='object')

In [107]:
# Which years are included?
df['year'].unique()

array([ 1948.,  1952.,  1954.,  1956.,  1958.,  1960.,  1962.,  1964.,
        1966.,  1968.,  1970.,  1972.,  1974.,  1976.,  1978.,  1980.,
        1982.,  1984.,  1986.,  1988.,  1990.,  1992.,  1994.,  1996.,
        1998.,  2000.,  2002.])

In [112]:
#let's do the rest with one specific year.
df1992 = df[['year', 'age', 'gender', 'income', 'vote', 'presvote_intent']] \
    .loc[lambda df:df['year'] == 1992.0]

## look for nulls
df1992.isnull().sum(axis=0)

year                 0
age                  0
gender               0
income             156
vote               177
presvote_intent     72
dtype: int64

In [113]:
df1992.shape

(1947, 6)

In [114]:
# drop Nans
df1992 = df1992.dropna(axis = 0)
df1992.shape,df1992.head()

((1591, 6),
          year age     gender                  income           vote  \
 32092  1992.0  41  2. female  4. 68 to 95 percentile  2. yes, voted   
 32093  1992.0  83  2. female  2. 17 to 33 percentile  2. yes, voted   
 32095  1992.0  67  2. female   1. 0 to 16 percentile  2. yes, voted   
 32096  1992.0  28    1. male  2. 17 to 33 percentile  2. yes, voted   
 32097  1992.0  71  2. female  3. 34 to 67 percentile  2. yes, voted   
 
                                 presvote_intent  
 32092  2. republican candidate (with or without  
 32093  2. republican candidate (with or without  
 32095  1. democratic candidate (with or without  
 32096  2. republican candidate (with or without  
 32097  1. democratic candidate (with or without  )

In [115]:
df1992['age_code'] = df1992['age'].astype(int)
df1992['income_code'] = df1992['income'].str.extract('(\d)', expand=False).astype(int)
df1992['gender_code'] = df1992['gender'].str.extract('(\d)', expand=False).astype(int)
df1992['presvote_intent_code'] = df1992['presvote_intent'].str.extract('(\d)', expand=False).astype(int).apply(lambda x: x-1)
#drop voters not d or r
df1992 = df1992.drop(df1992[df1992['presvote_intent_code'] > 1].index)
df1992['presvote_intent_code'].unique()

array([1, 0])

In [116]:
import statsmodels.api as sm
import numpy as np
from patsy import dmatrices
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

y,X = dmatrices('presvote_intent_code ~ income_code', data=df1992, return_type='dataframe')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
model1 = sm.Logit(y_train, X_train).fit()
model1.summary()

Optimization terminated successfully.
         Current function value: 0.663311
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                            
================================================================================
Dep. Variable:     presvote_intent_code   No. Observations:                 1138
Model:                            Logit   Df Residuals:                     1136
Method:                             MLE   Df Model:                            1
Date:                  Sun, 15 Oct 2017   Pseudo R-squ.:                 0.01975
Time:                          13:07:09   Log-Likelihood:                -754.85
converged:                         True   LL-Null:                       -770.05
                                          LLR p-value:                 3.497e-08
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -1.2838      0.182     -7.037      0.000      -1.641      -0.926
income_code     0.3005      0.056      5.409      0.000       0.192       0.409
===============================================================================
"""

In [117]:
y2,X2 = dmatrices('presvote_intent_code ~ income_code + gender_code + age_code', data=df1992, return_type='dataframe')
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=1)
model2 = sm.Logit(y2_train, X2_train).fit()
model2.summary()

Optimization terminated successfully.
         Current function value: 0.660772
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                            
================================================================================
Dep. Variable:     presvote_intent_code   No. Observations:                 1138
Model:                            Logit   Df Residuals:                     1134
Method:                             MLE   Df Model:                            3
Date:                  Sun, 15 Oct 2017   Pseudo R-squ.:                 0.02350
Time:                          13:07:48   Log-Likelihood:                -751.96
converged:                         True   LL-Null:                       -770.05
                                          LLR p-value:                 6.830e-08
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -1.3589      0.334     -4.064      0.000      -2.014      -0.704
income_code     0.3093      0.057      5.442      0.000       0.198       0.421
gender_code    -0.1842      0.123     -1.493      0.135      -0.426       0.058
age_code        0.0071      0.004      1.940      0.052   -7.27e-05       0.014
===============================================================================
"""